In [4]:
import tensorflow as tf
import numpy as np

from tensorflow.keras.applications import ResNet50V2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import Sequential
from tensorflow.keras import layers
from tensorflow.keras.models import load_model

import mlflow
from mlflow import pyfunc
import mlflow.tensorflow

In [5]:
train_dir = r'datasets\train'

In [6]:
def run_training(model=None):
    
    mlflow.tensorflow.autolog()
    
    with mlflow.start_run():
        if model is None:

            resnet = ResNet50V2(include_top=False, weights='imagenet')

            model = Sequential()
            model.add(resnet)
            model.add(layers.GlobalAveragePooling2D())
            model.add(layers.Dense(4096, activation='relu'))
            model.add(layers.Dense(120, activation='softmax'))

        train_datagen = ImageDataGenerator(samplewise_center=True, 
                                         samplewise_std_normalization=True,
                                         rotation_range=15, zoom_range=0.2,
                                         horizontal_flip=True,
                                         validation_split=0.2
                                        )

        train_generator = train_datagen.flow_from_directory(train_dir,
                target_size=(150, 150),
                batch_size=32,
                subset='training')

        validation_generator = train_datagen.flow_from_directory(train_dir,
                target_size=(150, 150),
                batch_size=32,
                subset='validation')

        model.build()
        optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)
        model.compile(optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

        for layer in resnet.layers[:-10]:
            layer.trainable = True

        model.fit(train_generator, steps_per_epoch=250, epochs=40,
                validation_data=validation_generator, validation_steps=validation_generator.samples // 32)
        
        pyfunc_model = pyfunc.load_model(mlflow.get_artifact_uri("model"))

        return pyfunc_model
    

In [7]:
model = run_training()

Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.


ValueError: Asked to retrieve element 0, but the Sequence has length 0